In [1]:
import numpy as np 
import pandas as pd 
import os

In [2]:
class_names = os.listdir('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/CINIC10/train')
len(class_names)

10

In [3]:
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/CINIC10/train',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1

In [4]:
len(images)

90000

In [5]:
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])
dataset = D
random.shuffle(dataset)

In [6]:
import torch
import torchvision
import torchvision.transforms as transforms
from torchvision import models
from torchsummary import summary
features = models.alexnet().features  # get the feature extraction layers
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
flatten = torch.nn.Flatten()# create a flatten layer
model = torch.nn.Sequential(features, flatten).to(device)  # create a new model with the flatten layer
summary(model)

Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0


Layer (type:depth-idx)                   Param #
├─Sequential: 1-1                        --
|    └─Conv2d: 2-1                       23,296
|    └─ReLU: 2-2                         --
|    └─MaxPool2d: 2-3                    --
|    └─Conv2d: 2-4                       307,392
|    └─ReLU: 2-5                         --
|    └─MaxPool2d: 2-6                    --
|    └─Conv2d: 2-7                       663,936
|    └─ReLU: 2-8                         --
|    └─Conv2d: 2-9                       884,992
|    └─ReLU: 2-10                        --
|    └─Conv2d: 2-11                      590,080
|    └─ReLU: 2-12                        --
|    └─MaxPool2d: 2-13                   --
├─Flatten: 1-2                           --
Total params: 2,469,696
Trainable params: 2,469,696
Non-trainable params: 0

In [7]:
from tqdm import tqdm
import cv2
def extract_features(train):
    features = []
    labels = []
    for i in tqdm(train,desc='loading...'):
        im = cv2.imread(i[0])
        im = im.astype('float32')
        im = cv2.resize(im, (224,224))
        im = torch.from_numpy(im)
        im = im.to(device)
        im = im.unsqueeze(0) # adding an extra dimension at index 0 which refers to the batch size =1
        im = im.permute(0, 3, 1, 2) # reorder the dimensions of the input tensor from [batch_size, height, width, num_channels] to [batch_size, num_channels, height, width]. 
        with torch.no_grad():
            outputs = model(im)
            features.append(outputs.cpu().numpy()[0])
            labels.append(i[1])
    return np.array(features), np.array(labels)

In [8]:
x_train, y_train = extract_features(dataset)

loading...: 100%|██████████| 90000/90000 [25:08<00:00, 59.65it/s]


In [10]:
y_train.reshape(90000,1)
c = np.column_stack((x_train, y_train)) 
df = pd.DataFrame(c)
df.to_csv('CINIC_train_features.csv')

In [11]:
class_names = os.listdir('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/CINIC10/test')
images = [] #Get path to all images and save them in a list
labels = [] # labels of corresponding images
class_id = 0

for i in class_names:
    dir=os.path.join('C:/Users/ritti/OneDrive/Desktop/MT2022090,MT2022162/CINIC10/test',i)
    x = [os.path.join(dir, f) for f in os.listdir(dir)]
    images+= x
    labels+=[class_id]*len(x)
    class_id+=1
    
import random
D = []
for i in range(len(images)):
    D.append([images[i],labels[i]])
    
    
dataset = D
random.shuffle(dataset)

x_test, y_test = extract_features(dataset)
y_test.reshape(len(images),1)

c = np.column_stack((x_test, y_test))
df = pd.DataFrame(c)
df.to_csv('CINIC_test_features.csv')

loading...: 100%|██████████| 90000/90000 [30:44<00:00, 48.78it/s]  


In [12]:
from sklearn import svm
clf = svm.SVC(C=1.8, random_state=42,verbose =2)
clf.fit(x_train, y_train)

[LibSVM]

SVC(C=1.8, random_state=42, verbose=2)

In [14]:
import pickle
filename = 'CINIC_10.sav'
pickle.dump(clf, open(filename, 'wb'))

In [15]:
from sklearn.metrics import accuracy_score
pred_labels = clf.predict(x_test)
accuracy = accuracy_score(y_test, pred_labels)
print('Accuracy:', accuracy)

TESTING

In [1]:
import pickle
loaded_model = pickle.load(open('CINIC_10.sav', 'rb'))

In [2]:
import pandas as pd
df = pd.read_csv('CINIC_test_features.csv')

In [3]:
df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216
0,0,1.920312,2.171518,2.311523,2.022948,2.179287,2.264148,1.972996,2.521769,2.311523,...,0.0,0.0,0.0,3.004256,0.0,0.0,0.0,0.0,0.0,6.0
1,1,1.189102,1.189102,1.786231,1.739729,2.206394,2.087101,1.189102,1.426312,1.786231,...,0.0,0.0,0.0,2.573787,0.0,0.0,0.0,0.0,0.0,3.0
2,2,1.888230,1.888230,2.050573,1.880832,1.332951,2.201527,1.888230,1.909231,2.050573,...,0.0,0.0,0.0,2.776734,0.0,0.0,0.0,0.0,0.0,9.0
3,3,0.043233,0.043233,0.134851,0.977557,1.089261,0.884210,0.600005,0.600005,1.311795,...,0.0,0.0,0.0,1.291077,0.0,0.0,0.0,0.0,0.0,6.0
4,4,1.168355,1.979221,1.699420,1.699420,1.636407,1.917034,1.168355,2.534917,2.534917,...,0.0,0.0,0.0,2.138705,0.0,0.0,0.0,0.0,0.0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89995,89995,1.606338,1.778440,1.827244,2.109880,2.171946,2.234909,1.606338,1.984633,1.984633,...,0.0,0.0,0.0,1.882725,0.0,0.0,0.0,0.0,0.0,5.0
89996,89996,0.925481,1.210614,1.585515,1.265289,1.393550,0.815506,2.238951,2.259101,2.099452,...,0.0,0.0,0.0,1.794839,0.0,0.0,0.0,0.0,0.0,5.0
89997,89997,1.628936,1.916122,1.691305,1.691305,1.476266,1.389608,1.813441,1.916122,2.181581,...,0.0,0.0,0.0,1.816679,0.0,0.0,0.0,0.0,0.0,4.0
89998,89998,1.278403,1.278403,1.751806,2.135399,2.135399,2.333463,1.278403,1.794995,1.751806,...,0.0,0.0,0.0,1.008605,0.0,0.0,0.0,0.0,0.0,3.0


In [4]:
# shuffle the DataFrame rows
df = df.sample(frac = 1)

In [5]:
data=df.iloc[:, 1:]
data

,0,1,2,3,4,5,6,7,8,9,...,9207,9208,9209,9210,9211,9212,9213,9214,9215,9216
79028,1.721566,1.721566,1.588128,1.613694,1.613694,1.209439,1.807933,2.099534,1.919024,1.902720,...,0.0,0.0,0.0,1.084467,0.0,0.0,0.0,0.0,0.0,1.0
13988,1.892386,2.119595,1.991541,2.036136,1.646198,1.824443,1.970160,2.484720,2.286500,2.643130,...,0.0,0.0,0.0,1.815848,0.0,0.0,0.0,0.0,0.0,4.0
64349,1.565099,1.565099,1.299154,1.353760,1.585160,2.043866,2.018088,2.018088,1.696862,1.696862,...,0.0,0.0,0.0,2.018836,0.0,0.0,0.0,0.0,0.0,4.0
68314,0.576795,0.576795,0.619264,0.691173,0.550821,1.116363,0.576795,0.576795,0.668161,0.691173,...,0.0,0.0,0.0,1.135041,0.0,0.0,0.0,0.0,0.0,0.0
49176,0.958824,0.958824,0.793583,0.746769,0.834645,1.155286,1.240017,1.240017,0.979313,0.892838,...,0.0,0.0,0.0,1.546458,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75941,1.130031,1.130031,2.051142,1.711344,1.509766,1.276409,1.653608,1.653608,2.432836,2.432836,...,0.0,0.0,0.0,1.551497,0.0,0.0,0.0,0.0,0.0,7.0
38313,2.483642,2.675917,2.252922,2.055696,2.213347,2.880595,2.483642,2.675917,2.252922,2.055696,...,0.0,0.0,0.0,2.495748,0.0,0.0,0.0,0.0,0.0,5.0
49256,2.710717,2.710717,2.134480,2.235799,2.512054,2.095499,2.779910,2.779910,2.359723,2.235799,...,0.0,0.0,0.0,2.138398,0.0,0.0,0.0,0.0,0.0,6.0
53408,1.387267,1.387267,2.042357,1.791032,1.509500,2.336644,1.387267,1.387267,2.042357,1.791032,...,0.0,0.0,0.0,2.359177,0.0,0.0,0.0,0.0,0.0,1.0


In [13]:
x_test = data.iloc[0:1000,0:9216].to_numpy()
y_test = data.iloc[0:1000,9216].to_numpy()

In [14]:
x_test.shape

(1000, 9216)

In [15]:
y_test.shape

(1000,)

In [16]:
from sklearn.metrics import accuracy_score
pred_labels = loaded_model.predict(x_test)
accuracy = accuracy_score(y_test, pred_labels)
print('Accuracy:', accuracy)

Accuracy: 0.488
